TODO: New here:
- add filechooser
- ...

# Setup

## Setup masci-tools path

In [1]:
# IMPORTANT: we need to import stuff from masci-tools folder.
# Since masci-tools is not installed as a module (yet), the notebook kernel
# needs to be started in the masci-tools folder.
# If that has not happened for some reason, then need to add the masci-tools
# manually to the sys path.
import os
import sys

cwd = os.getcwd()
path_mtools = cwd
dirname_mtools = "masci-tools"
# first try if we can get away without needing an absolute path
if dirname_mtools in path_mtools:
    while os.path.basename(path_mtools) != dirname_mtools:
        path_mtools = os.path.split(path_mtools)[0]
else:
    # okay, try with an absolute path
    path_mtools = "/home/johannes/Desktop/Studium/Kurse_RWTH/SiScLab/18W/repos/masci-tools"
    if not os.path.isdir(path_mtools):
        raise IOError(f"Could not find path to masci-tools. Please specify absolute path.")

# found masci-tools. add to syspath (for imports) and chdir.
if path_mtools not in sys.path:
    # add only once
    sys.path.append(path_mtools)

## Setup imports

In [2]:
# Jupyter, Python imports
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
import traitlets
from tkinter import Tk, filedialog

#  python 3interactive figures in a live IPython notebook session
# if run from jupyter-notebook: MAGICmatplotlib nbagg
# if run from jupyter-lab: MAGICmatplotlib widget
%matplotlib widget

# studentproject18ws imports
import logging
from studentproject18w.hdf.reader import Reader
from studentproject18w.hdf.recipes import Recipes
from studentproject18w.plot.matplot import BandDOSPlot, BandPlot

In [3]:
# reloading mpl backend ensures interactive plot works first time.
# At least on my system, I need to load it twice so that it works.
%matplotlib widget

## Read file, import data

- TODO: link with widget-based file-chooser

In [4]:
# # NO DOS file:
filename = 'banddos.hdf'
filenames_dos = []
# filename = 'banddos_4x4.hdf'
# filenames_dos = []
# filename = 'banddos_sodium.hdf'
# filenames_dos = []

# 1 DOS file:
# filename = os.path.join('MoSe2', 'banddos_2spin.hdf')
# filenames_dos = [os.path.join('MoSe2',"DOS.1")]

# # 2 DOS files:
# filename = os.path.join('Co', 'banddos_Co.hdf')
# filenames_dos = [os.path.join('Co', 'DOS.1'), os.path.join('Co', 'DOS.2')]

filepath = ['..', 'data', 'input', filename]
filepath = os.path.join(*filepath)
filepaths_dos = [['..', 'data', 'input', fd] for fd in filenames_dos]
filepaths_dos = [os.path.join(*fpd) for fpd in filepaths_dos]

data = None
reader = Reader(filepath=filepath)
with reader as h5file:
    data = reader.read(recipe=Recipes.FleurBands)
    #
    # Note:
    # Inside the with statement (context manager),
    # all data attributes that are type h5py Dataset are available (in-file access)
    # When the statement is left,the HDF5 file gets closed and the datasets are closed.
    #
    # Use data outside the with-statement (in-memory access: all HDF5 datasets converted to numpy ndarrays):
    data.move_datasets_to_memory()

## Init data <--> plot interface

In [5]:
# init interface plot <--> data
plter = BandDOSPlot(plt, data, filepaths_dos)

# Define Widgets

In [6]:
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True)

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"

In [7]:
my_button = SelectFilesButton()
display(my_button)

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [8]:
my_button.files

[]

### Define user input arguments: band plot

In [9]:
select_ylim = widgets.FloatRangeSlider(
    value=plter.icdv.ylim.initial,
    min=plter.icdv.ylim.min,
    max=plter.icdv.ylim.max+1,
    step=plter.icdv.ylim.step,
    description=plter.icdv.ylim.label,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout=widgets.Layout(width="95%")
)

# number of bands can be large, so use a rangeslider instead of selectionslider
# select_bands_height = f"{len(plter.icdv.bands)*5}px"
select_bands = widgets.IntRangeSlider(
    value=plter.icdv.bands_slider.initial,
    min=plter.icdv.bands_slider.min, 
    max=plter.icdv.bands_slider.max, 
    step=plter.icdv.bands_slider.step,                                    
    description=plter.icdv.bands_slider.label,                              
    disabled=False, 
    continuous_update=False,                                    
    orientation='horizontal',
    readout=True,                                   
    readout_format='d',
    layout=select_ylim.layout
)

select_exponent = widgets.FloatSlider(
    value=plter.icdv.exponent.initial,
    min=plter.icdv.exponent.min,
    max=plter.icdv.exponent.max,
    step=plter.icdv.exponent.step,
    description=plter.icdv.exponent.label,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout=select_ylim.layout
)

select_marker_size = widgets.FloatSlider(
    value=plter.icdv.marker_size.initial,
    min=plter.icdv.marker_size.min,
    max=plter.icdv.marker_size.max,
    step=plter.icdv.marker_size.step,
    description=plter.icdv.marker_size.label,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout=select_ylim.layout
)


select_compare_characters = widgets.Checkbox(
    value=False,
    description="",
    disabled=True,
#     style={'description_width': 'initial'} # for long descriptions
#     layout=widgets.Layout(width='30%', height='30%')
)

select_characters = widgets.SelectMultiple(
    options=plter.icdv.characters,
    value=tuple(plter.icdv.characters),
    description='',
    disabled=False,
#     layout=widgets.Layout(width='120%', height='90%')
)

# select_groups_height = f"{len(plter.icdv.groups)*19}px" # height just so no scrollbar needed
# select_groups_height = f"{len(plter.icdv.groups)*10}px"
select_groups_options = [(label,value) for label,value 
                         in zip(plter.icdv.group_labels, plter.icdv.groups)]
select_groups = widgets.SelectMultiple(
    options=select_groups_options,
    value=tuple(plter.icdv.groups),
    description='',                                  
    disabled=False,
#     layout=widgets.Layout(width='20%', height='100%')
)


select_ignore_apg = widgets.Checkbox(
    value=False,
    description="Ignore atoms per group",
    disabled=False
)

select_spins = widgets.SelectMultiple(
    options=plter.icdv.spins,
#     value=(tuple(plter.icdv.groups)),
    value=tuple([0]),
    description='',
    disabled=False
)

### Define user input arguments: DOS plot
TODO: these only are visible if (a) DOS file(s) are loaded.

In [10]:
select_dos_groups = widgets.Checkbox(
    value=True,
    description="DOS groups",
    disabled=False
)

select_dos_interstitial = widgets.Checkbox(
    value=True,
    description="DOS interstitial",
    disabled=False
)

select_dos_characters = widgets.Checkbox(
    value=False,
    description="DOS all characters",
    disabled=False
)

### Connect interdependent widgets

In [11]:
# disable compare_characters when not 2 characters are selected
def on_character_selection_change(change):
    if len(change.new) == 2 and (len(select_spins.value) != 2):
        select_compare_characters.disabled = False
#         select_compare_characters.description = "Compare 2 characters (enabled)"
    else:
        select_compare_characters.disabled = True
#         select_compare_characters.description = "Compare 2 characters (disabled)"
select_characters.observe(on_character_selection_change, names='value')

# disable compare_characters when 2 spins are selected.
# it would work (plot just uses the down spin then), but would be confusing.
def on_spin_selection_change(change):
    if len(change.new) == 2:
        select_compare_characters.disabled = True
#         select_compare_characters.description = "Compare 2 characters (enabled)"
    elif (len(select_characters.value) == 2):
        select_compare_characters.disabled = False
#         select_compare_characters.description = "Compare 2 characters (disabled)"
select_spins.observe(on_spin_selection_change, names='value')

# disable select_characters and select_spins when compare_characters is active, 
def on_compare_character(change):
    select_characters.disabled = change.new
    select_spins.disabled = change.new
select_compare_characters.observe(on_compare_character, names='value')

### Define Layout containers

TODO: integrate new widgets:
- spin

In [12]:
layout_auto = widgets.Layout(width='auto') #override default layouts
def override_layout(widgetss):
    for widget in widgetss:
        no_description=(widget.description=="")
        widget.layout=layout_auto
        if no_description:
            widget.description=''
        
def hbox_widget_label(wig, lab, wig_left=True):
    if wig_left:
        return widgets.HBox([wig,lab])
    else:
        return widgets.HBox([lab,wig])
#     wig.layout=layout_auto
#     lab.layout=layout_auto
#     hbox_layout = widgets.Layout(
#         display='flex',
#         flex_flow='row',
#         justify_content='space-between'
#     ) # this is def of HBox I think
#     children = None
#     if wig_left:
#         children = [wig,lab]
#     else:
#         children = [lab,wig]
#     box = widgets.Box(
#         children=children,
#         layout=hbox_layout
#     )
#     return box

selects_slider = [
    select_ylim,
    select_bands,
    select_exponent,
    select_marker_size
]
override_layout(selects_slider)
layout_box_sliders = widgets.Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    border='solid',
    width='60%'
)
box_sliders = widgets.Box(
    children=selects_slider,
    layout=layout_box_sliders
)


label_characters = widgets.Label("Characters")
label_compare_characters = widgets.Label("Compare 2")
selects_character = [
    label_characters,
    select_characters,
    label_compare_characters,
    select_compare_characters
]
override_layout(selects_character)
select_characters.description=''
# selects_character.append(hbox_widget_label(
#     select_compare_characters, 
#     label_compare_characters, wig_left=False))
layout_box_characters = widgets.Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    border='solid',
    width='20%'
)
box_characters = widgets.Box(
    children=selects_character,
    layout=layout_box_characters
)


label_groups = widgets.Label("Atom Groups")
selects_group = [
    label_groups,
    select_groups
]
override_layout(selects_group)
select_groups.description=''
box_groups = widgets.Box(
    children=selects_group,
    layout=layout_box_characters
)

dashboard_controls = widgets.HBox([box_sliders, box_characters, box_groups])
# display(dashboard_controls)

In [13]:
# init plot
fig_scale = 0.65
fig_ratio = [12,6]
figsize=[fig_scale * el for el in fig_ratio]

(fig, ax_bands, ax_dos) = plter.setup_figure(fig_ratio, fig_scale)
# (fig2, ax_bands2) = plter2.setup_figure(fig_ratio, fig_scale)


plt.suptitle(f"Bandstructure of {filename}")


def update_plot(characters, groups, bands, spins,
                unfolding_weight_exponent, marker_size,
               compare_characters, ylim, ignore_atoms_per_group,
               dos_groups, dos_interstitial, dos_characters):
    
    (mask_bands, mask_characters, mask_groups) = plter.icdv.convert_selections(
        bands, characters, groups)
    

    plter.plot_bandDOS(mask_bands, mask_characters, mask_groups, spins,
                          unfolding_weight_exponent, compare_characters, 
                              ignore_atoms_per_group, marker_size,
                              dos_groups, dos_interstitial, dos_characters,
                               dos_fix_xlim=True, ylim=ylim)

        
    plt.show()

    
interactive_update_plot = interactive(
    update_plot, 
    characters=select_characters, groups=select_groups, 
    bands=select_bands, spins=select_spins,
    unfolding_weight_exponent=select_exponent, marker_size=select_marker_size,
    compare_characters=select_compare_characters, ylim=select_ylim,
    ignore_atoms_per_group=select_ignore_apg,
    dos_groups=select_dos_groups,
    dos_interstitial=select_dos_interstitial,
    dos_characters=select_dos_characters
)


# dashboard = widgets.VBox()
# dashboard_plot = widgets.VBox(interactive_update_plot.children)
# dashboard_plot.layout.display='none'
# dashboard.children = [dashboard_plot,
#                      dashboard_controls]

select_characters.description=''
select_groups.description=''
select_compare_characters.description=''
display(dashboard_controls)

# TODO: integrate into dashboard:
display(select_ignore_apg)
display(select_spins)

# TODO: integrate into dashboard:
display(select_dos_groups)
display(select_dos_interstitial)
display(select_dos_characters)

FigureCanvasNbAgg()

Checkbox(value=False, description='Ignore atoms per group')

SelectMultiple(description='spins', index=(0,), options=(0,), value=(0,))

Checkbox(value=True, description='DOS groups')

Checkbox(value=True, description='DOS interstitial')

Checkbox(value=False, description='DOS all characters')

## 3D Atoms plot

In [21]:
import ipyvolume as ipv
x, y, z = data.atoms_position.T
@interact(groups=select_groups)
def update_atoms_plot(groups):
    selected_atoms = []
    for group in groups:
        selected_atoms_group = np.where(data.atoms_group==group)
        for selected_atom_group in selected_atoms_group:
            selected_atoms.extend(selected_atom_group)
    
    ipv.figure()
    scatter = ipv.scatter(x,y,z, size=5, marker="sphere", selected=selected_atoms, size_selected=8)
    ipv.show()

    

interactive(children=(SelectMultiple(description='groups', index=(2,), layout=Layout(width='auto'), options=((…

In [20]:
data.atoms_elements

[Si, Si, Si, Si, C, Si, Si, Si, Si, Si, Si, Si, Si, Si, Si, Si]